# 1. Import library

In [1]:
import pandas as pd
import csv
from numpy import dstack, hstack
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from numpy import mean
from numpy import std
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import time

import spidev
from numpy import interp
from time import sleep
import RPi.GPIO as GPIO
import signal
import datetime, time
# from threading import _Timer
import csv
import termios,sys,tty, fcntl, os,select
import threading
from sklearn import preprocessing

# 2. Load Dataset

In [2]:
def load_dataset():
  # data_filenames = ['data_0v3_2.csv','data_1v3_2.csv','data_2v3_2.csv','data_3v3_2.csv','data_7v3_2.csv','data_8v3_2.csv',
                    # 'data_9v3_2.csv','data_4v3_3.csv','data_5v3_3.csv','data_6v3_3.csv']
  data_filenames = ['data_0_v5_3.csv','data_1_v5_3.csv','data_2_v5_3.csv','data_3_v5_3.csv','data_4_v5_3.csv','data_5_v5_3.csv','data_6_v5_3.csv','data_7_v5_3.csv'
  ,'data_8_v5_3.csv','data_9_v5_3.csv']
  photodiode_arr = ['6','1','2','3','9','8','7','4','5']
  data_frame = []
  for i,file in enumerate (data_filenames):
    data = pd.read_csv(file)
    data_frame.append (data)
  data_frame = pd.concat(data_frame, join = 'inner')
  data_3D = list()
  for i, photodiode_name in enumerate(photodiode_arr):
    data_3D.append(data_frame[photodiode_name].values.reshape(-1, ))  
  data_3D = dstack(data_3D)
  data_3D = data_3D.reshape((-1, 500,9))
  data_y = pd.read_csv('label_v5.csv')
  result_none = np.where(data_y == -1)
  data_y = data_y.drop([data_y.index[260],   data_y.index[891],   data_y.index[892]], axis = 0)
  data_3D = np.delete(data_3D, [260, 891, 892 ], axis = 0)
  data_y = to_categorical(data_y,num_classes=10)
  Xtra,Xval,Ytra,Yval = train_test_split(data_3D,data_y,test_size=0.2,shuffle=True)
  return Xtra, Ytra, Xval, Yval


# 3. Build the model

In [4]:

def build_model(): 
  # model.add(GRU(150, input_shape=(n_timesteps,n_features), return_sequences=True))
  # model.add(GRU(50))
  model.add(GRU(80, input_shape=(n_timesteps,n_features), return_sequences=False))
  # model.add(LSTM(80))
  # model.add(Dropout(0.2))
  # model.add(GRU(150))
  # model.add(Bidirectional( GRU(100))) 
  # model.add(Dropout(0.5))
  # model.add(Dense(100, activation='relu'))
  model.add(Dense(80, activation='relu'))
  model.add(Dense(n_outputs, activation='softmax'))
  model.summary()


# 4. Training the model

In [5]:
def evaluate_model(trainX, trainy, testX, testy):
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model_checkpoint= ModelCheckpoint('RNN_model_weights.hdf5', save_best_only=True, monitor='val_loss', 
                                      mode='auto', save_weights_only=True)
        # fit network
        history = model.fit(trainX, trainy, epochs=epochs, validation_split=0.2, batch_size=batch_size, verbose=verbose)
        # evaluate model
        _, accuracy = model.evaluate(testX, testy,batch_size=batch_size, verbose=0)
        return accuracy, history

def summarize_results(scores):
        print(scores)
        m, s = mean(scores), std(scores)
        print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

def run_experiment(repeats=10):
   
        build_model()
        
        scores = list()
        for r in range(repeats):
            score, history = evaluate_model(trainX, trainy, testX, testy)
            score = score * 100.0
            print('>#%d: %.3f' % (r+1, score))
            scores.append(score)
#             plt.plot(history.history['loss'])
#             plt.plot(history.history['val_loss'])
#             plt.title('Training Loss VS Validation Loss')
#             plt.ylabel('Loss')
#             plt.xlabel('epoch')
#             plt.legend(['train', 'val'], loc='upper left')
#             plt.show()
        # summarize results
        summarize_results(scores)

# 5. Initialize the SPI and TERMIO

In [2]:
def spi_init():
        spi.open(0,0)
        spi.max_speed_hz =1350000
        spi2.open(0,1)
        spi2.max_speed_hz =1350000
        
def termios_init():
        newattr[3] = newattr[3] & ~termios.ICANON & ~termios.ECHO
        termios.tcsetattr(fd, termios.TCSANOW, newattr)
        fcntl.fcntl(fd, fcntl.F_SETFL, oldflags | os.O_NONBLOCK)

# 6. Timer Class

In [3]:
class RepeatedTimer(object):
  def __init__(self, interval, function, *args, **kwargs):
    self._timer = None
    self.interval = interval
    self.function = function
    self.args = args
    self.kwargs = kwargs
    self.is_running = False
    self.next_call = time.time()
    self.start()

  def _run(self):
    self.is_running = False
    self.start()
    self.function(*self.args, **self.kwargs)

  def start(self):
    if not self.is_running:
      self.next_call += self.interval
      self._timer = threading.Timer(self.next_call - time.time(), self._run)
      self._timer.start()
      self.is_running = True

  def stop(self):
    self._timer.cancel()
    self.is_running = False

# 7. Read Sensor Data

In [4]:
def get_char():
    fd = sys.stdin.fileno()
    old_settings = termios.tcgetattr(fd)
    try:
        tty.setraw(sys.stdin.fileno())
        ch = sys.stdin.read(1)
    finally:
        termios.tcsetattr(fd, termios.TCSADRAIN, old_settings)
    return ch


def analogInput_CE0(channel,spi):
    adc = spi.xfer2([1, (8+channel)<<4, 0])
    data = ((adc[1]&3) <<8) + adc[2]
    return data

def analogInput_CE1(channel,spi2):
    adc = spi2.xfer2([1, (8+channel)<<4, 0])
    data = ((adc[1]&3) <<8) + adc[2]
    return data

def read_data():
        if len(output)  < 500:
            aux = []
#             aux.append(time.time())
            aux.append(analogInput_CE1(2,spi2))
            aux.append(analogInput_CE0(3,spi))
            aux.append(analogInput_CE0(7,spi))
            aux.append(analogInput_CE1(3,spi2))
            aux.append(analogInput_CE1(1,spi2))
            aux.append(analogInput_CE0(5,spi))
            aux.append(analogInput_CE0(1,spi))
            aux.append(analogInput_CE0(2,spi))
            aux.append(analogInput_CE0(6,spi))
            '''aux.append(analogInput_CE0(0,spi))
            aux.append(analogInput_CE0(1,spi))
            aux.append(analogInput_CE0(2,spi))
            aux.append(analogInput_CE0(3,spi))
            aux.append(analogInput_CE1(0,spi2))
            aux.append(analogInput_CE1(1,spi2))
            aux.append(analogInput_CE1(2,spi2))
            aux.append(analogInput_CE1(3,spi2))
            aux.append(analogInput_CE1(4,spi2))'''
            output.append(aux)
#         print('output is : ', output)
    

# def get_real_data():
#         i = 0
#         real_data = []
#         while i < 500:
#             aux = []
# #             aux.append(time.time())
#             aux.append(analogInput_CE0(0,spi))
#             aux.append(analogInput_CE0(1,spi))
#             aux.append(analogInput_CE0(2,spi))
#             aux.append(analogInput_CE0(3,spi))
#             aux.append(analogInput_CE1(0,spi2))
#             aux.append(analogInput_CE1(1,spi2))
#             aux.append(analogInput_CE1(2,spi2))
#             aux.append(analogInput_CE1(3,spi2))
#             aux.append(analogInput_CE1(4,spi2))
#     #         print('from the CE1_C5: ',aux)
#             real_data.append(aux)
#             i+=1

#         return real_data_3D
        

In [9]:
    # Training model
prev_time = time.time()
verbose, epochs, batch_size, num_Neurons = 0, 40, 128, 100
trainX, trainy, testX, testy = load_dataset()
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]    
model = Sequential() 
run_experiment()
print('Interval is : ', time.time() - prev_time)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 80)                21840     
_________________________________________________________________
dense (Dense)                (None, 80)                6480      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 29,130
Trainable params: 29,130
Non-trainable params: 0
_________________________________________________________________
>#1: 100.000
>#2: 100.000
>#3: 100.000
>#4: 100.000
>#5: 100.000
>#6: 100.000
>#7: 100.000
>#8: 100.000
>#9: 100.000
>#10: 100.000
[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
Accuracy: 100.000% (+/-0.000)
Interval is :  12833.48143696785


In [5]:
from keras.models import save_model, load_model
# filepath = './saved_model'
# save_model(model, filepath, save_format='h5')
model = load_model('saved_model')

In [ ]:
if __name__ == '__main__':

    
    
    global spi, spi2, output,fd, oldterm, newattr, oldflags, real_data_3D
    spi = spidev.SpiDev()
    spi2 = spidev.SpiDev()
    output = []
#     fd = sys.stdin.fileno()
#     oldterm = termios.tcgetattr(fd)
#     oldflags = fcntl.fcntl(fd, fcntl.F_GETFL)
#     newattr = termios.tcgetattr(fd)
    min_max_scaler = preprocessing.MinMaxScaler(feature_range = (0, 100), copy = True)
    spi_init()
#     termios_init()
    RepeatedTimer(0.01,read_data).start()
    while 1:
        inkey = input()
        if inkey == 'n':
            print('start get input')
            output = []
            sleep(5.5)
            output_3D = []
            output_3D.append(min_max_scaler.fit_transform(output[0:500]))
            real_data_3D = np.array(output_3D)
#             real_data_3D = min_max_scaler.fit_transform()
#             print(real_data_3D.shape)
            real_data_3D.reshape((-1, 500, 9))
#             print(real_data_3D)
            label = model.predict(real_data_3D)
#             print("label is ", label)
            print('label is ', np.argmax(label))

n
start get input
label is  3
n
start get input
label is  0
n
start get input
label is  0
n
start get input
label is  0
n
start get input
label is  9
n
start get input
label is  0
m
n
start get input
label is  0
n
start get input
label is  9
n
start get input
label is  4
n
start get input
label is  9
n
start get input
label is  0
n
start get input
label is  0
n
start get input
label is  4
n
start get input
label is  4
n
start get input
label is  9
n
start get input
label is  0
n
start get input
label is  4
n
start get input
label is  0
n
start get input
label is  0
n
start get input
label is  4
n
start get input
label is  4
n
start get input
label is  4
n
start get input
label is  0
n
start get input
label is  0
n
start get input
label is  4
n
start get input
label is  0
n
start get input
label is  0
n
start get input
label is  4
n
start get input
label is  4
n
start get input
label is  4
n
start get input
label is  4
n
start get input
label is  6
n
start get input
label is  6
n
start 

In [1]:
a = input()

2
